In [1]:
from __future__ import division, print_function, absolute_import

seed_num = 12

import numpy as np
np.random.seed(seed_num)
import random
random.seed(seed_num)
import tensorflow as tf
tf.set_random_seed(seed_num)

import os
import shutil
import sys
import glob
import math
from functools import reduce
import operator as op
from sklearn.utils import class_weight
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support

dataset_folder = os.path.abspath("./individual_npzs_two_seconds_overlap/{0}/*.npz")
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

np.set_printoptions(suppress=True)

## Parameters

In [2]:
learning_rate = 0.01
batch_size = 64
dropout = 0.5
max_pool = 2
strides = 1
input_size = 40000
output_size = 4
epochs = 1000
timesteps = 25
first_kernel = [5, 5, 1, 16]
second_kernel = [5, 5, 16, 24]
third_kernel = [5, 5, 24, 32]

hidden_layer = 128
dense_layer = 64
l2_beta = 0.01

validation_session = 1
test_session = 2

label_dictionary = {'ang': 0, 'hap': 1, 'neu': 2, 'sad': 3}
onehot_dictionary = {0: 'ang', 1: 'hap', 2:'neu', 3:'sad'}

## Helper methods

In [3]:
def even_batch_generator(data_dictionary):
    steps = math.ceil(data.shape[0] / batch_size)
    counts = [math.floor(class_count / steps) for class_count in np.unique(train_labels, return_counts=True)[1].tolist()]
    for batch_step in range(0, steps):
        minibatch = []
        if (batch_step != steps-1):
            for count in counts:
                while (count != 0): 
                    minibatch.append()
    
    
def batch_generator(data, labels):
    steps = math.ceil(data.shape[0] / batch_size)
    for batch_step in range(0, steps):
        start = batch_size * batch_step
        end = batch_size * (batch_step + 1)
        yield data[start:end], labels[start:end], batch_step
        
def build_encoded_array(emotion_label):
    initialized_array = [0. for key in label_dictionary]
    initialized_array[label_dictionary[emotion_label]] = 1.
    return initialized_array
        
def onehot_encode(label_minibatch):
    return [build_encoded_array(emotion_label) for emotion_label in label_minibatch]

def compute_class_weights(labels):
    return np.ndarray.tolist(class_weight.compute_class_weight('balanced', np.unique(labels), labels))

def sparse_encode(label_minibatch):
    return [label_dictionary[emotion_label] for emotion_label in label_minibatch]

def onehot_decode(onehot_labels):
    return [onehot_dictionary[label.index(1.0)] for label in onehot_labels]

def weighted_accuracy(y_pred, y_true):
    #Suma vsetkych spravne urcenych viet predelena vsetkymi vetami

    correct = 0
    all = 0
    for key in y_true:
        correct += y_pred[key]
        all += y_true[key]

    return correct/all


def unweighted_accuracy(y_pred, y_true):

    #Suma poctu spravnych viet lomeno vsetky vety pre kazdu emociu (triedu) predelena poctom tried

    correct = 0
    emotions = len(y_true)
    for key in y_true:
        correct += y_pred[key]/y_true[key]

    return correct/emotions

def validation_results(current_sess, data, labels):
    predictions = current_sess.run(correct_prediction, feed_dict={X:data, Y:labels, keep_prob:1.0, normalization_switch: False})
    coupled_validation = list(zip(predictions, onehot_decode(labels)))
    final_dict = {'ang':0, 'hap':0, 'neu':0, 'sad':0}
    overall_dict = {'ang':0, 'hap':0, 'neu':0, 'sad':0}

    for couple in coupled_validation:
        if (couple[0]):
            final_dict[couple[1]] += 1
        overall_dict[couple[1]] += 1
    unweighted_acc = unweighted_accuracy(final_dict, overall_dict)
    return unweighted_acc

## Load dataset: 4 sessions for training, 1 for test

In [4]:
train_dataset = []
validation_dataset = []
test_dataset = []

train_labels = []
validation_labels = []
test_labels = []

all_sessions = {1: [], 2:[], 3:[], 4:[], 5:[]}

session_string = 'session{0}'

for i in range(1, 6):
    formatted = session_string.format(i)
    for spectrogram in glob.glob(dataset_folder.format(formatted)):
        loaded_spec = np.load(spectrogram)
        for x in loaded_spec['spectrograms']:
            if i != validation_session and i != test_session:
                train_dataset.append(x) 
            elif i == validation_session:
                validation_dataset.append(x)
            elif i == test_session:
                test_dataset.append(x)
        for x in loaded_spec['labels']:
            all_sessions[i].append(x)
            if i != validation_session and i != test_session:
                train_labels.append(x) 
            elif i == validation_session:
                validation_labels.append(x)
            elif i == test_session:
                test_labels.append(x)
        
train_dataset = np.asarray(train_dataset)
train_labels = np.asarray(train_labels)

validation_dataset = np.asarray(validation_dataset)
validation_labels = np.asarray(validation_labels)

test_dataset = np.asarray(test_dataset)
test_labels = np.asarray(test_labels)

In [5]:
train_data = np.zeros([len(train_dataset), train_dataset[0].shape[0], train_dataset[0].shape[1]], dtype=np.float32)
for data in range(len(train_dataset)):
    train_data[data,:,:] = train_dataset[data]
    
validation_data = np.zeros([len(validation_dataset), validation_dataset[0].shape[0], validation_dataset[0].shape[1]], dtype=np.uint8)
for data in range(len(validation_dataset)):
    validation_data[data,:,:] = validation_dataset[data]
    
test_data = np.zeros([len(test_dataset), test_dataset[0].shape[0], test_dataset[0].shape[1]], dtype=np.float32)
for data in range(len(test_dataset)):
    test_data[data,:,:] = test_dataset[data]

In [6]:
counts = np.unique(train_labels, return_counts=True)[1]
sum_counts = sum(counts)
class_weights = [(0.25/(count/sum_counts)) for count in counts]

In [7]:
class_weights

[1.6739788199697427, 1.8818027210884352, 0.6059693318729463, 0.819022945965951]

In [8]:
np.unique(validation_labels, return_counts=True)

(array(['ang', 'hap', 'neu', 'sad'], dtype='<U3'),
 array([180,  93, 519, 374], dtype=int64))

In [9]:
np.unique(test_labels, return_counts=True)

(array(['ang', 'hap', 'neu', 'sad'], dtype='<U3'),
 array([ 90, 150, 796, 371], dtype=int64))

In [10]:
np.unique(train_labels, return_counts=True)

(array(['ang', 'hap', 'neu', 'sad'], dtype='<U3'),
 array([ 661,  588, 1826, 1351], dtype=int64))

In [11]:
for session in all_sessions:
    print(session, np.unique(all_sessions[session], return_counts=True))

1 (array(['ang', 'hap', 'neu', 'sad'], dtype='<U3'), array([180,  93, 519, 374], dtype=int64))
2 (array(['ang', 'hap', 'neu', 'sad'], dtype='<U3'), array([ 90, 150, 796, 371], dtype=int64))
3 (array(['ang', 'hap', 'neu', 'sad'], dtype='<U3'), array([246, 173, 536, 539], dtype=int64))
4 (array(['ang', 'hap', 'neu', 'sad'], dtype='<U3'), array([302, 125, 481, 370], dtype=int64))
5 (array(['ang', 'hap', 'neu', 'sad'], dtype='<U3'), array([113, 290, 809, 442], dtype=int64))


In [12]:
np.set_printoptions(threshold=np.nan)
train_data[0]

array([[  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0., 

In [13]:
validation_data = validation_data.reshape((validation_data.shape[0], input_size))
validation_labels = onehot_encode(validation_labels)

test_data = test_data.reshape((test_data.shape[0], input_size))
test_labels = onehot_encode(test_labels)

train_data = train_data.reshape((train_data.shape[0], input_size))
train_labels = onehot_encode(train_labels)

In [14]:
train_data, train_labels = shuffle(train_data, train_labels, random_state=seed_num)

## Pipeline

In [15]:
def conv2d(to_process, weights, biases, batchnorm_switch, strides=1):
    conv_out = tf.nn.conv2d(to_process, weights, strides=[1, strides, strides, 1], padding='SAME')
    bias_out = tf.nn.bias_add(conv_out, biases)
    batchnorm_out = tf.layers.batch_normalization(bias_out, training=batchnorm_switch)
    relu_out = tf.nn.leaky_relu(batchnorm_out)
    return relu_out

def maxpool2d(to_pool, pool_size=2):
    maxpool_out = tf.nn.max_pool(to_pool, ksize=[1, pool_size, pool_size, 1], strides=[1, pool_size, pool_size, 1], padding='SAME')
    return maxpool_out

def nn_pipeline_rnn(spectrogram, weights, biases, dropout_num, normalization_switch):
    reshaped_input = tf.reshape(spectrogram, shape=[-1, 200, 200, 1])

    first_layer_out = conv2d(reshaped_input, weights['first_layer_weights'], biases['first_layer_biases'], normalization_switch)
    first_maxpool_out = maxpool2d(first_layer_out, pool_size=2)

    second_layer_out = conv2d(first_maxpool_out, weights['second_layer_weights'], biases['second_layer_biases'], normalization_switch)
    second_maxpool_out = maxpool2d(second_layer_out, pool_size=2)

    third_layer_out = conv2d(second_maxpool_out, weights['third_layer_weights'], biases['third_layer_biases'], normalization_switch)
    third_maxpool_out = maxpool2d(third_layer_out, pool_size=2)

    interim_shape = third_maxpool_out.get_shape().as_list()
    transposed = tf.transpose(third_maxpool_out, perm=[0, 2, 1, 3])
    reshape_for_rnn = tf.reshape(transposed, [-1, interim_shape[2], interim_shape[1]*interim_shape[3]])
    reshape_for_rnn.set_shape([None, interim_shape[2], interim_shape[1]*interim_shape[3]])
    
    fw_cell = tf.contrib.rnn.GRUCell(hidden_layer)
    bw_cell = tf.contrib.rnn.GRUCell(hidden_layer)
    gru_fw_cell = tf.contrib.rnn.DropoutWrapper(cell=fw_cell, output_keep_prob=dropout_num, seed=(seed_num+32))
    gru_bw_cell = tf.contrib.rnn.DropoutWrapper(cell=bw_cell, output_keep_prob=dropout_num, seed=(seed_num+64))
    
    gru_output, _ = tf.nn.bidirectional_dynamic_rnn(gru_fw_cell, gru_bw_cell, reshape_for_rnn, dtype=tf.float32)
    gru_output_concatted = tf.concat(gru_output, axis=2)
    interim_shape_gru = tf.shape(gru_output_concatted)
    gru_flatten = tf.reshape(gru_output_concatted, [-1, interim_shape_gru[1]*interim_shape_gru[2]])
    
    fully_connected_out = tf.add(tf.matmul(gru_flatten, weights['gru_weights']), biases['gru_biases'])
    fully_connected_activation = tf.nn.leaky_relu(fully_connected_out)
    fully_connected_dropout = tf.nn.dropout(fully_connected_activation, dropout_num, seed=seed_num)
    
    prediction = tf.add(tf.matmul(fully_connected_dropout, weights['output']), biases['output'])
        
    return prediction

In [16]:
weights_rnn = {
    'first_layer_weights': tf.Variable(tf.truncated_normal(first_kernel, stddev=np.sqrt(2 / first_kernel[0]), seed=seed_num)),
    'second_layer_weights': tf.Variable(tf.truncated_normal(second_kernel, stddev=np.sqrt(2 / second_kernel[0]),seed=(seed_num+4))),
    'third_layer_weights': tf.Variable(tf.truncated_normal(third_kernel, stddev=np.sqrt(2 / third_kernel[0]),seed=(seed_num+8))),
    'gru_weights': tf.Variable(tf.truncated_normal([2*hidden_layer*timesteps, dense_layer], stddev=np.sqrt(2 / (2*hidden_layer*timesteps)), seed=(seed_num+2))),
    'output': tf.Variable(tf.truncated_normal([dense_layer, output_size], stddev=np.sqrt(2 / dense_layer), seed=(seed_num+16)))
}

biases_rnn = {
    'first_layer_biases': tf.Variable(tf.zeros([16])),
    'second_layer_biases': tf.Variable(tf.zeros([24])),
    'third_layer_biases': tf.Variable(tf.zeros([32])),
    'gru_biases': tf.Variable(tf.zeros([dense_layer])),
    'output': tf.Variable(tf.zeros([output_size]))
}

In [17]:
def loss_function(logits, labels):
    flat_logits = tf.reshape(logits, [-1, output_size])
    flat_labels = tf.reshape(labels, [-1, output_size])
    
    eps = tf.constant(value=1e-10)
    flat_logits = flat_logits + eps
    
    softmax = tf.nn.softmax(flat_logits)
    
    coeffs = tf.constant(class_weights)
    
    cross_entropy = -tf.reduce_sum(tf.multiply(flat_labels * tf.log(softmax + eps), coeffs), reduction_indices=[1])
    
    l2_loss = tf.add_n([tf.nn.l2_loss(weights_rnn[weights]) for weights in weights_rnn]) * l2_beta
    
    return tf.reduce_mean(tf.add(cross_entropy, l2_loss))

In [18]:
X = tf.placeholder(tf.float32, [None, input_size])
Y = tf.placeholder(tf.float32, [None, output_size])

keep_prob = tf.placeholder(tf.float32)
normalization_switch = tf.placeholder(tf.bool)

logits = nn_pipeline_rnn(X, weights_rnn, biases_rnn, keep_prob, normalization_switch)
prediction = tf.nn.softmax(logits)

#weighted_logits = tf.multiply(class_weights, logits)
#weight_regularizer = tf.add_n([tf.nn.l2_loss(weights_rnn[weights]) for weights in weights_rnn]) * 0.01
#loss_function = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))

flat_logits = tf.reshape(logits, [-1, output_size])
flat_labels = tf.reshape(Y, [-1, output_size])
eps = tf.constant(value=1e-10)
flat_logits = flat_logits + eps
softmax = tf.nn.softmax(flat_logits)
coeffs = tf.constant(class_weights)
cross_entropy = -tf.reduce_sum(tf.multiply(flat_labels * tf.log(softmax + eps), coeffs), reduction_indices=[1])
l2_loss = tf.add_n([tf.nn.l2_loss(weights_rnn[weights]) for weights in weights_rnn]) * l2_beta
loss_function = tf.reduce_mean(tf.add(cross_entropy, l2_loss))

optimizer = tf.train.AdadeltaOptimizer(learning_rate=learning_rate)
extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(extra_update_ops):
    train_trigger = optimizer.minimize(loss_function)

correct_prediction = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

init = tf.global_variables_initializer()

saver = tf.train.Saver()

In [19]:
with open('./Test/train_metadata.txt'.format(test_session), 'w') as metadata_file:
    metadata_string = "Learning rate: {0}\nBatch size: {1}\nDropout: {2}\nEpochs: {3}\nHidden layer: {4}\nL2 beta: {5}\nTest session: {6}\n".format(learning_rate, batch_size, dropout, epochs, hidden_layer, l2_beta, test_session)
    metadata_file.write(metadata_string)

In [42]:
def train_network(session):
    
    best_acc = 0
    best_loss = sys.maxsize
    best_train_acc = 0
    current_tolerance = 0
    
    for epoch_step in range(1, epochs+1):
        batch_gen = batch_generator(train_data, train_labels)
        train_acc_all = []
        train_loss_all = []
        for data_minibatch, label_minibatch, current_index in batch_gen:
            sess.run(train_trigger, feed_dict={X: data_minibatch, Y: label_minibatch, keep_prob: dropout, normalization_switch: True})
            train_loss, train_acc = sess.run([loss_function, accuracy], feed_dict={X: data_minibatch, Y: label_minibatch, keep_prob: 1.0, normalization_switch: False})
            #print("Train loss and acc of batch {0}: {1}, {2}".format(current_index, train_loss, train_acc))
            train_acc_all.append(train_acc)
            train_loss_all.append(train_loss)
        train_acc_whole = sum(train_acc_all)/len(train_acc_all)
        train_loss_whole = sum(train_loss_all)/len(train_loss_all)
        acc_loss_file = open("./Test/train_loss_logs.txt".format(test_session), "a+")
        acc_loss_file.write("{0}, {1}, {2}\n".format(epoch_step, train_acc_whole, train_loss_whole))
        acc_loss_file.close()
        print("Training accuracy and loss of epoch #" + str(epoch_step) + ": {:.4f}".format(train_acc_whole) + ", {:.4f}".format(train_loss_whole))
        loss, acc = sess.run([loss_function, accuracy], feed_dict={X: validation_data, Y: validation_labels, keep_prob: 1.0, normalization_switch: False})
        predics = sess.run(tf.argmax(prediction, 1), feed_dict={X: validation_data, Y: validation_labels, keep_prob: 1.0, normalization_switch: False})
        conf = confusion_matrix(predics, [onehot.index(1.0) for onehot in validation_labels])
        print(conf)
        validation_recall = np.mean(np.diag(conf) / np.sum(conf, axis=1))
        unweighted_acc = validation_results(sess, validation_data, validation_labels)
        print("Validation after epoch #" + str(epoch_step) + ", Validation Loss= "+ "{:.4f}".format(loss) + ", Validation Accuracy= " + "{:.3f}".format(acc) + ", Unweighted Accuracy= " + "{:.3f}".format(unweighted_acc) + ", Recall= " + " {:.3f}".format(validation_recall))

        if train_acc_whole > best_train_acc:
            best_train_acc = train_acc_whole
            saver.save(sess, "./Test/acc/overall/best_model_accuracy.ckpt".format(test_session))
            print("Saved model by train acc {0}".format(train_acc_whole))
        
        if train_loss_whole < best_loss:
            best_loss = train_loss_whole
            saver.save(sess, "./Test/loss/overall/best_model_loss.ckpt".format(test_session))
            print("Saved model by train loss {0}".format(train_loss_whole))
        if epoch_step % 20 == 0:
            saver.save(sess, "./Test/acc/{1}/best_model_epoch.ckpt".format(test_session, epoch_step, epoch_step))
            


    print("Optimization Finished!")

In [21]:
#with tf.device('/gpu:0'):
#    sess = tf.Session()
#    sess.run(init)
    
#    print('Session initialized.')
#    train_network(sess)

    #print("Testing Accuracy:", \
    #    sess.run(accuracy, feed_dict={X: test_data,
    #                                  Y: test_labels,
    #                                  keep_prob: 1.0, normalization_switch: False}))

In [22]:
#sess.close()

In [ ]:
with tf.device('/gpu:0'):
    sess = tf.Session()

    saver.restore(sess, "./Test/acc/overall/best_model_accuracy.ckpt")    
    train_network(sess)

INFO:tensorflow:Restoring parameters from ./Test/acc/overall/best_model_accuracy.ckpt
Training accuracy and loss of epoch #1: 0.9636, 12.3735
[[100  11  28   9]
 [  9  17  44  12]
 [ 39  44 191  55]
 [ 32  21 256 298]]
Validation after epoch #1, Validation Loss= 13.8157, Validation Accuracy= 0.520, Unweighted Accuracy= 0.476, Recall=  0.489
Saved model by train acc 0.9636160714285714
Saved model by train loss 12.373512186322893
Training accuracy and loss of epoch #2: 0.9629, 12.3630
[[ 94  10  27   9]
 [  8  16  41  10]
 [ 45  45 195  56]
 [ 33  22 256 299]]
Validation after epoch #2, Validation Loss= 13.8339, Validation Accuracy= 0.518, Unweighted Accuracy= 0.467, Recall=  0.487
Saved model by train loss 12.36296945299421
Training accuracy and loss of epoch #3: 0.9638, 12.3493
[[ 96  10  27   9]
 [ 10  21  45  11]
 [ 37  38 179  50]
 [ 37  24 268 304]]
Validation after epoch #3, Validation Loss= 13.8274, Validation Accuracy= 0.515, Unweighted Accuracy= 0.479, Recall=  0.497
Saved mode

Saved model by train loss 12.080751895904541
Training accuracy and loss of epoch #27: 0.9694, 12.0687
[[ 95  10  27   9]
 [  8  16  39  10]
 [ 42  43 195  57]
 [ 35  24 258 298]]
Validation after epoch #27, Validation Loss= 13.5935, Validation Accuracy= 0.518, Unweighted Accuracy= 0.468, Recall=  0.489
Saved model by train loss 12.068707289014544
Training accuracy and loss of epoch #28: 0.9699, 12.0572
[[ 98  11  27   9]
 [  8  17  43  11]
 [ 39  41 188  52]
 [ 35  24 261 302]]
Validation after epoch #28, Validation Loss= 13.5741, Validation Accuracy= 0.519, Unweighted Accuracy= 0.474, Recall=  0.491
Saved model by train loss 12.057214178357805
Training accuracy and loss of epoch #29: 0.9688, 12.0464
[[ 99  11  29  10]
 [  8  17  42  11]
 [ 39  41 188  54]
 [ 34  24 260 299]]
Validation after epoch #29, Validation Loss= 13.5623, Validation Accuracy= 0.517, Unweighted Accuracy= 0.474, Recall=  0.488
Saved model by train loss 12.046376582554409
Training accuracy and loss of epoch #30: 0.

Saved model by train acc 0.9738839285714286
Saved model by train loss 11.788188757215227
Training accuracy and loss of epoch #53: 0.9732, 11.7772
[[ 97  10  28  10]
 [  8  19  45  11]
 [ 42  40 185  51]
 [ 33  24 261 302]]
Validation after epoch #53, Validation Loss= 13.3533, Validation Accuracy= 0.517, Unweighted Accuracy= 0.477, Recall=  0.492
Saved model by train loss 11.777212687901088
Training accuracy and loss of epoch #54: 0.9723, 11.7664
[[ 98  10  31  10]
 [  8  18  44  11]
 [ 41  41 183  52]
 [ 33  24 261 301]]
Validation after epoch #54, Validation Loss= 13.3412, Validation Accuracy= 0.515, Unweighted Accuracy= 0.474, Recall=  0.486
Saved model by train loss 11.766448416028704
Training accuracy and loss of epoch #55: 0.9737, 11.7550
[[ 99  11  31  10]
 [  9  19  46  12]
 [ 38  39 181  51]
 [ 34  24 261 301]]
Validation after epoch #55, Validation Loss= 13.3181, Validation Accuracy= 0.515, Unweighted Accuracy= 0.477, Recall=  0.487
Saved model by train loss 11.755030850001743

Saved model by train loss 11.50659841809954
Training accuracy and loss of epoch #79: 0.9792, 11.4958
[[ 95  10  30   8]
 [  8  16  41  12]
 [ 43  44 193  53]
 [ 34  23 255 301]]
Validation after epoch #79, Validation Loss= 13.1357, Validation Accuracy= 0.519, Unweighted Accuracy= 0.469, Recall=  0.486
Saved model by train acc 0.9792410714285714
Saved model by train loss 11.495753996712821
Training accuracy and loss of epoch #80: 0.9788, 11.4850
[[ 94  10  28   6]
 [  8  16  38  13]
 [ 44  45 200  56]
 [ 34  22 253 299]]
Validation after epoch #80, Validation Loss= 13.1279, Validation Accuracy= 0.522, Unweighted Accuracy= 0.470, Recall=  0.491
Saved model by train loss 11.48500428880964
Training accuracy and loss of epoch #81: 0.9788, 11.4748
[[ 95  10  30   9]
 [  9  16  42  13]
 [ 41  43 189  51]
 [ 35  24 258 301]]
Validation after epoch #81, Validation Loss= 13.1185, Validation Accuracy= 0.515, Unweighted Accuracy= 0.467, Recall=  0.483
Saved model by train loss 11.474773761204311
T

Training accuracy and loss of epoch #105: 0.9808, 11.2252
[[ 88  10  25   6]
 [  9  16  35  12]
 [ 44  42 199  54]
 [ 39  25 260 302]]
Validation after epoch #105, Validation Loss= 12.9750, Validation Accuracy= 0.519, Unweighted Accuracy= 0.463, Recall=  0.493
Saved model by train loss 11.22517307826451
Training accuracy and loss of epoch #106: 0.9810, 11.2150
[[ 93  10  30   7]
 [  7  16  31  10]
 [ 44  43 197  54]
 [ 36  24 261 303]]
Validation after epoch #106, Validation Loss= 12.9457, Validation Accuracy= 0.522, Unweighted Accuracy= 0.470, Recall=  0.496
Saved model by train loss 11.215024771009173
Training accuracy and loss of epoch #107: 0.9819, 11.2044
[[ 95  10  30   7]
 [  9  18  41  12]
 [ 41  42 197  56]
 [ 35  23 251 299]]
Validation after epoch #107, Validation Loss= 12.8979, Validation Accuracy= 0.522, Unweighted Accuracy= 0.475, Recall=  0.493
Saved model by train acc 0.9819196428571428
Saved model by train loss 11.204387719290597
Training accuracy and loss of epoch #10

...

Saved model by train loss 8.754073592594692
Training accuracy and loss of epoch #397: 0.9951, 8.7474
[[102  13  27  12]
 [  7  16  41  12]
 [ 42  44 221  61]
 [ 29  20 230 289]]
Validation after epoch #397, Validation Loss= 11.0607, Validation Accuracy= 0.539, Unweighted Accuracy= 0.484, Recall=  0.496
Saved model by train loss 8.74744690486363
Training accuracy and loss of epoch #398: 0.9953, 8.7405
[[ 98  11  25   9]
 [  8  15  36  12]
 [ 41  44 208  57]
 [ 33  23 250 296]]
Validation after epoch #398, Validation Loss= 11.0981, Validation Accuracy= 0.529, Unweighted Accuracy= 0.474, Recall=  0.496
Saved model by train loss 8.740506213051932
Training accuracy and loss of epoch #399: 0.9953, 8.7334
[[ 99  13  26  10]
 [  8  15  36  11]
 [ 42  43 215  61]
 [ 31  22 242 292]]
Validation after epoch #399, Validation Loss= 11.0942, Validation Accuracy= 0.533, Unweighted Accuracy= 0.477, Recall=  0.494
Saved model by train loss 8.733412333897181
Training accuracy and loss of epoch #400: 0.9

Saved model by train loss 8.571546990530832
Training accuracy and loss of epoch #424: 0.9953, 8.5646
[[ 96  13  28  11]
 [  8  15  36  13]
 [ 42  43 200  50]
 [ 34  22 255 300]]
Validation after epoch #424, Validation Loss= 11.0085, Validation Accuracy= 0.524, Unweighted Accuracy= 0.471, Recall=  0.486
Saved model by train loss 8.564600018092564
Training accuracy and loss of epoch #425: 0.9953, 8.5581
[[ 98  13  26  11]
 [  8  16  42  14]
 [ 41  42 210  55]
 [ 33  22 241 294]]
Validation after epoch #425, Validation Loss= 10.9523, Validation Accuracy= 0.530, Unweighted Accuracy= 0.477, Recall=  0.491
Saved model by train loss 8.558057771410262
Training accuracy and loss of epoch #426: 0.9951, 8.5516
[[103  13  29  12]
 [  7  14  37  12]
 [ 40  44 209  55]
 [ 30  22 244 295]]
Validation after epoch #426, Validation Loss= 10.9660, Validation Accuracy= 0.533, Unweighted Accuracy= 0.479, Recall=  0.489
Saved model by train loss 8.551556655338832
Training accuracy and loss of epoch #427: 0.

Saved model by train loss 8.392032527923584
Training accuracy and loss of epoch #451: 0.9951, 8.3857
[[ 96  12  25   9]
 [  7  15  39  13]
 [ 41  43 211  57]
 [ 36  23 244 295]]
Validation after epoch #451, Validation Loss= 10.8696, Validation Accuracy= 0.529, Unweighted Accuracy= 0.472, Recall=  0.493
Saved model by train loss 8.385688877105713
Training accuracy and loss of epoch #452: 0.9951, 8.3794
[[ 98  13  27  12]
 [  7  16  38  12]
 [ 40  41 200  46]
 [ 35  23 254 304]]
Validation after epoch #452, Validation Loss= 10.8679, Validation Accuracy= 0.530, Unweighted Accuracy= 0.479, Recall=  0.494
Saved model by train loss 8.379387038094658
Training accuracy and loss of epoch #453: 0.9953, 8.3726
[[ 95  12  23  10]
 [  7  16  40  13]
 [ 42  42 210  54]
 [ 36  23 246 297]]
Validation after epoch #453, Validation Loss= 10.8357, Validation Accuracy= 0.530, Unweighted Accuracy= 0.475, Recall=  0.496
Saved model by train loss 8.37257971082415
Training accuracy and loss of epoch #454: 0.9

Saved model by train loss 8.219723578861782
Training accuracy and loss of epoch #478: 0.9958, 8.2138
[[104  15  28  14]
 [  7  17  45  16]
 [ 38  41 209  52]
 [ 31  20 237 292]]
Validation after epoch #478, Validation Loss= 10.6497, Validation Accuracy= 0.533, Unweighted Accuracy= 0.486, Recall=  0.491
Saved model by train loss 8.213778414045061
Training accuracy and loss of epoch #479: 0.9955, 8.2076
[[ 93  11  24   8]
 [  7  16  40  13]
 [ 41  41 199  44]
 [ 39  25 256 309]]
Validation after epoch #479, Validation Loss= 10.7497, Validation Accuracy= 0.529, Unweighted Accuracy= 0.475, Recall=  0.499
Saved model by train loss 8.207554871695383
Training accuracy and loss of epoch #480: 0.9955, 8.2014
[[102  13  25  11]
 [  7  16  43  16]
 [ 40  44 209  56]
 [ 31  20 242 291]]
Validation after epoch #480, Validation Loss= 10.6555, Validation Accuracy= 0.530, Unweighted Accuracy= 0.480, Recall=  0.492
Saved model by train loss 8.201362555367606
Training accuracy and loss of epoch #481: 0.

Saved model by train loss 8.05358097893851
Training accuracy and loss of epoch #505: 0.9953, 8.0477
[[ 94  12  24   9]
 [  7  14  40  14]
 [ 42  43 209  52]
 [ 37  24 246 299]]
Validation after epoch #505, Validation Loss= 10.6057, Validation Accuracy= 0.528, Unweighted Accuracy= 0.469, Recall=  0.490
Saved model by train loss 8.047738252367292
Training accuracy and loss of epoch #506: 0.9958, 8.0416
[[ 89   9  22   6]
 [  8  17  42  14]
 [ 44  44 208  53]
 [ 39  23 247 301]]
Validation after epoch #506, Validation Loss= 10.6142, Validation Accuracy= 0.527, Unweighted Accuracy= 0.471, Recall=  0.501
Saved model by train loss 8.041622338976179
Training accuracy and loss of epoch #507: 0.9955, 8.0356
[[104  14  28  14]
 [  7  15  39  13]
 [ 40  45 217  56]
 [ 29  19 235 291]]
Validation after epoch #507, Validation Loss= 10.5310, Validation Accuracy= 0.538, Unweighted Accuracy= 0.484, Recall=  0.491
Saved model by train loss 8.035587692260743
Training accuracy and loss of epoch #508: 0.9

In [20]:
sess = tf.Session()

saver.restore(sess, "./Test/acc/overall/best_model_accuracy.ckpt".format(test_session))

INFO:tensorflow:Restoring parameters from ./Test/acc/overall/best_model_accuracy.ckpt


In [33]:
def get_confusion_matrix(session, data, labels):
    predictions = sess.run(tf.argmax(prediction, 1), feed_dict={X:data, Y:labels, keep_prob:1.0, normalization_switch: False})
    true_labels = [onehot.index(1.0) for onehot in labels]
    return confusion_matrix(predictions, true_labels)

def get_final_results(session, data, labels):
    final_dict = {'ang': 0, 'hap': 0, 'neu': 0, 'sad': 0}
    overall_dict = {'ang': 0, 'hap': 0, 'neu': 0, 'sad': 0}
    predictions = session.run(correct_prediction, feed_dict={X:data, Y:labels, keep_prob:1.0, normalization_switch: False})
    coupled = list(zip(predictions, onehot_decode(labels)))
    
    for couple in coupled:
        if (couple[0]):
            final_dict[couple[1]] += 1
        overall_dict[couple[1]] += 1
    
    return final_dict, overall_dict

def get_final_results_glued(session, labels, correct):
    final_dict = {'ang': 0, 'hap': 0, 'neu': 0, 'sad': 0}
    overall_dict = {'ang': 0, 'hap': 0, 'neu': 0, 'sad': 0}
    coupled = list(zip(correct, onehot_decode(labels)))
    
    for couple in coupled:
        if (couple[0]):
            final_dict[couple[1]] += 1
        overall_dict[couple[1]] += 1
    
    return final_dict, overall_dict

In [34]:
final_preds = np.asarray([])
final_correct_preds = np.asarray([])
final_probs = np.empty((0, 4))
final_probs_softmax = np.empty((0, 4))


midpoint = math.ceil(len(test_labels)/2)
for i in range(0, 2):
    predictions_test = sess.run(correct_prediction, feed_dict={X:test_data[i*midpoint:(i+1)*midpoint], Y:test_labels[i*midpoint:(i+1)*midpoint], keep_prob:1.0, normalization_switch: False})
    probabilities_test = sess.run(prediction, feed_dict={X:test_data[i*midpoint:(i+1)*midpoint], Y:test_labels[i*midpoint:(i+1)*midpoint], keep_prob:1.0, normalization_switch: False})
    predictions_test_confusion = sess.run(tf.argmax(prediction, 1), feed_dict={X:test_data[i*midpoint:(i+1)*midpoint], Y:test_labels[i*midpoint:(i+1)*midpoint], keep_prob:1.0, normalization_switch: False})
    predictions_test_softmax = sess.run(prediction, feed_dict={X:test_data[i*midpoint:(i+1)*midpoint], Y:test_labels[i*midpoint:(i+1)*midpoint], keep_prob:1.0, normalization_switch: False})

    final_correct_preds = np.concatenate((final_correct_preds, predictions_test))
    final_probs = np.concatenate((final_probs, probabilities_test))
    final_preds = np.concatenate((final_preds, predictions_test_confusion))
    final_probs_softmax = np.concatenate((final_probs_softmax, predictions_test_softmax))

In [35]:
confusion_matrix(final_preds, [onehot.index(1.0) for onehot in test_labels])

array([[ 45,  14,  69,   9],
       [ 12,  55, 115,  18],
       [ 29,  51, 401,  46],
       [  4,  30, 211, 298]], dtype=int64)

In [36]:
conf = confusion_matrix(final_preds, [onehot.index(1.0) for onehot in test_labels])
np.mean(np.diag(conf) / np.sum(conf, axis=0))

0.5434175030588703

In [37]:
final_dict_test, overall_dict_test = get_final_results_glued(sess, test_labels, final_correct_preds)

In [38]:
print(final_dict_test)
print(overall_dict_test)

{'ang': 45, 'hap': 55, 'neu': 401, 'sad': 298}
{'ang': 90, 'hap': 150, 'neu': 796, 'sad': 371}


In [39]:
print(unweighted_accuracy(final_dict_test, overall_dict_test))
print(weighted_accuracy(final_dict_test, overall_dict_test))

0.5434175030588703
0.5678749111584932


In [40]:
print(precision_recall_fscore_support(final_preds, [onehot.index(1.0) for onehot in test_labels], average='macro')[0:2])

(0.5434175030588703, 0.47829522895423815)


In [41]:
final_probs_softmax

array([[0.12588641, 0.64194512, 0.1941472 , 0.03802124],
       [0.01897654, 0.01669154, 0.49182105, 0.4725109 ],
       [0.00121475, 0.00809743, 0.42964131, 0.56104648],
       [0.99889982, 0.00001259, 0.001039  , 0.0000485 ],
       [0.99635649, 0.000001  , 0.00364122, 0.00000136],
       [0.99520677, 0.00004292, 0.00473156, 0.00001876],
       [0.9901399 , 0.00014903, 0.00958551, 0.00012545],
       [0.95984721, 0.01040282, 0.02511021, 0.00463966],
       [0.82104397, 0.00323055, 0.14941029, 0.02631508],
       [0.93339723, 0.00005945, 0.06634589, 0.00019751],
       [0.89043707, 0.0220263 , 0.07302211, 0.01451456],
       [0.99153239, 0.00023945, 0.00511895, 0.00310923],
       [0.99762827, 0.00006219, 0.00122245, 0.00108712],
       [0.83007276, 0.02474378, 0.14353669, 0.00164682],
       [0.11559814, 0.06444629, 0.77456677, 0.04538875],
       [0.70746785, 0.01894205, 0.18863599, 0.08495409],
       [0.8420307 , 0.00180001, 0.02945332, 0.12671596],
       [0.72229594, 0.00885915,